# PySpark en Google Colab

Instalacion Octubre/2021

1.   Instalacion Java
2.   Instalacion de Spark
3.   Instalar PySpark

## De forma General para usar pyspark en Colab (2021) siga los siguientes pasos en una celda en Colab:

In [2]:
# instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
# Descargar la ultima versión de java ( comprobar que existen los path de descarga)
# Download latest release. Update if necessary

!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
%ls -la /content/



In [5]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
%ls -la

In [ ]:
# instalar pyspark
!pip install -q pyspark

# Variables de entorno

In [8]:
import os # libreria de manejo del sistema operativo
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

# Cargar pyspark en el sistema

In [ ]:
from pyspark.sql import SparkSession

APP_NAME = "PDGE-tutorialSpark1"
SPARK_URL = "local[*]"
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
spark

Pruebas


In [11]:
sc = spark.sparkContext
#obtener el contexto de ejecución de Spark del Driver.
sc

<SparkContext master=local[*] appName=PDGE-tutorialSpark1>

In [ ]:
array = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 2)
array

In [ ]:
print(array.collect())

In [ ]:
print(array.count())


In [15]:
num3 = array.map(lambda elemento: 3*elemento)

In [ ]:
print(num3.collect())

# Ejemplos de operaciones con RDDs de Spark

In [ ]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10],2)

print(numeros.reduce(lambda e1,e2: e1+e2))

In [ ]:
pnumeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

rdd = pnumeros.map(lambda e: 2*e)

print(rdd.collect())


## Pregunta TS1.1 ¿Cómo hacer para obtener una lista de los elementos al cuadrado?

In [ ]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

rdd = numeros.filter(lambda elemento: elemento%2==0)

print (rdd.collect())

## Pregunta TS1.2 ¿Cómo filtrar los impares?

In [ ]:
rddi = numeros.filter (lambda e: e%2==1)
print (rddi.collect())

#Operaciones con números

Sumar n numeros

In [ ]:
rddnum=sc.range(1000000)
rddnum.reduce(lambda a,b: a+b)

In [ ]:
numeros = sc.parallelize([1,1,2,2,5])

unicos = numeros.distinct()

print (unicos.collect())

In [ ]:
numeros = sc.parallelize([1,2,3,4,5])

rdd = numeros.flatMap(lambda elemento: [elemento, 10*elemento])

print (rdd.collect())

Muestreo

In [ ]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

rdd = numeros.sample(True, 1.0)
# True = sin reemplazar
# 1.0 = fracción de elementos = 100%
print (rdd.collect())

Union 

In [ ]:
pares = sc.parallelize([2,4,6,8,10])
impares = sc.parallelize([1,3,5,7,9])

numeros = pares.union(impares)

print (numeros.collect())

reducción


In [ ]:
numeros = sc.parallelize([1,2,3,4,5])
# operación asociativa y conmutativa
# orden no está definido
print (numeros.reduce(lambda elem1,elem2: elem2+elem1))

## Pregunta TS1.3 ¿Tiene sentido esta operación? ¿Si se repite se obtiene siempre el mismo resultado?

In [ ]:
#Tiene sentido esta operación?
numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

In [ ]:
#Tiene sentido esta operación?
numeros = sc.parallelize([1,2,4,3,5])

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

#Acciones

In [ ]:
numeros = sc.parallelize([5,3,2,1,4])

print (numeros.take(3))
#¿Qué sucede si ponemos 30 en vez de 3 elementos?

In [ ]:
numeros = sc.parallelize([3,2,1,4,5])
# 
print (numeros.takeOrdered(3, lambda elem: -elem))
# La función lambda se está utilizando para crear el índice de la lista de ordenación 

## Pregunta TS1.4 ¿Cómo lo ordenarías para que primero aparezcan los impares y luego los pares?

# RDDs con string

In [ ]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_minus = palabras.map(lambda elemento: elemento.lower())

print (pal_minus.collect())

In [ ]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_long = palabras.map(lambda elemento: len(elemento))

print (pal_long.collect())

In [ ]:
log = sc.parallelize(['E: e21', 'W: w12', 'W: w13', 'E: e45'])

errors = log.filter(lambda elemento: elemento[0]=='E')

print (errors.collect())

In [ ]:
lineas = sc.parallelize(['', 'a', 'a b', 'a b c'])

palabras = lineas.flatMap(lambda elemento: elemento.split())

print (palabras.collect())

In [ ]:
lineas = sc.parallelize(['', 'a', 'a b', 'a b c'])

palabras_flat = lineas.flatMap(lambda elemento: elemento.split())
palabras_map = lineas.map(lambda elemento: elemento.split())

print (palabras_flat.collect())
print (palabras_map.collect())


## Pregunta TS1.5 ¿Cuántos elementos tiene cada rdd? ¿Cuál tiene más?


## Pregunta TS1.6 ¿De qué tipo son los elementos del rdd palabras_map? ¿Por qué palabras_map tiene el primer elemento vacío?

In [ ]:
palabras_flat.take(1)

In [ ]:
palabras_map


## Pregunta TS1.7. Prueba la transformación distinct si lo aplicamos a cadenas.

In [ ]:
log = sc.parallelize(['E: e21', 'I: i11', 'W: w12', 'I: i11', 'W: w13', 'E: e45'])

infos = log.filter(lambda elemento: elemento[0]=='I')
errors = log.filter(lambda elemento: elemento[0]=='E')

inferr = infos.union(errors)

print (inferr.collect())

## Pregunta TS1.8 ¿Cómo se podría obtener la misma salida pero utilizando una sola transformación y sin realizar la unión?

In [ ]:
numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem2+elem1))

In [ ]:
#Tiene sentido esta operación?
numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

In [ ]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_minus = palabras.map(lambda elemento: elemento.lower())

print (pal_minus.reduce(lambda elem1,elem2: elem1+"-"+elem2))
#y esta tiene sentido esta operación?
# Qué pasa si ponemos elem2+"-"+elem1

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr = r.reduceByKey(lambda v1,v2:v1+v2)
print (rr.collect())

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr1 = r.reduceByKey(lambda v1,v2:v1+v2)
print (rr1.collect())
rr2 = rr1.reduceByKey(lambda v1,v2:v1)
print (rr2.collect())


## Pregunta TS1.9 ¿Cómo explica el funcionamiento de las celdas anteriores?

In [ ]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr1 = r.reduceByKey(lambda v1,v2:'hola')
print (rr1.collect())
rr2 = rr1.reduceByKey(lambda v1,v2:'hola')
print (rr2.collect())

In [ ]:
r = sc.parallelize([('A', 1),('C', 2),('A', 3),('B', 4),('B', 5)])
rr = r.groupByKey()
res= rr.collect()
for k,v in res:
    print (k, list(v))
# Que operación realizar al RDD rr para que la operacion sea como un reduceByKey
#¿Y simular un group con un reduceByKey y un map?

In [ ]:
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('B',5),('C',6)])

rddjoin = rdd1.join(rdd2)

print (rddjoin.collect())
# Prueba a cambiar las claves del rdd1 y rdd2 para ver cuántos elementos se crean

In [ ]:
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('A',5),('B',6),('D',7)])

rddjoin = rdd1.join(rdd2)

print (rddjoin.collect())
#Modifica join por leftOuterJoin, rightOuterJoin y fullOuterJoin ¿Qué sucede? 


In [ ]:
rdd = sc.parallelize([('A',1),('B',2),('C',3),('A',4),('A',5),('B',6)])

res = rdd.sortByKey(False)

print (res.collect())

# Utilización de ficheros

In [43]:

# Crea una lista de 1000 números y la guarda.
# Da error si 'salida' existe, descomenta la linea de borrar en ese caso.
%rm -rf salida
numeros = sc.parallelize(range(0,1000))
numeros.saveAsTextFile('salida')


In [ ]:
ls -la

In [ ]:
%ls -la salida/*


In [ ]:
%cat salida/part-00000

In [ ]:
# Recupera el fichero guardado y realiza la suma
n2 = sc.textFile('salida').map(lambda a:int(a))
print(n2.reduce(lambda v1,v2: v1 + v2))

# Prueba este código y mira qué genera?
# Borra la salida y cambia las particiones en parallelize ¿Qué sucede? 
#    (pe c.parallelize(xrange(0,1000),8))


## Pregunta TS1.10 Borra la salida y cambia las particiones en parallelize ¿Qué sucede? 
  (pe c.parallelize(xrange(0,1000),8))


In [ ]:
%rm -rf salida
%ls -la


# El quijote

Montar el directorio de trabajo utilizando Google Drive
Loading Your Data into Google Colaboratory.
1. First of all, Upload your Data to your Google Drive.
2. Run the following script in colab shell.
3. Copy the authorization code of your account.
4. Paste the authorization code into the output shell.
5. Congrats! Now your Google Drive is mounted to this location /content/gdrive/MyDrive/




In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

In [ ]:
%ls /content/gdrive/MyDrive/

In [54]:
!cp "/content/gdrive/My Drive/elquijote.txt" .

In [ ]:
%ls -la

In [10]:
%pwd

'/content'

# Procesando el QUIJOTE

In [ ]:
quijote = sc.textFile("elquijote.txt")
quijote.take(10)

Transformaciones

In [63]:
charsPerLine = quijote.map(lambda s: len(s))
allWords = quijote.flatMap(lambda s: s.split())
allWordsNoArticles = allWords.filter(lambda a: a.lower() not in ["el", "la"])
allWordsUnique = allWords.map(lambda s: s.lower()).distinct()
sampleWords = allWords.sample(withReplacement=True, fraction=0.2, seed=666)
weirdSampling = sampleWords.union(allWordsNoArticles.sample(False, fraction=0.3))
# cómo funciona cada transformación

In [ ]:
allWordsUnique.take(10)

## Pregunta TS2.1 Explica la utilidad de cada transformación y detalle para cada una de ellas si cambia el número de elementos en el RDD resultante. Es decir si el RDD de partida tiene N elementos, y el de salida M elementos, indica si N>M, N=M o N<M.

* map
* flatmap
* filter
* distinct
* sample
* union


In [ ]:
numLines = quijote.count()
numChars = charsPerLine.reduce(lambda a,b: a+b) # also charsPerLine.sum()
sortedWordsByLength = allWordsNoArticles.takeOrdered(20, key=lambda x: -len(x))
numLines, numChars, sortedWordsByLength

## Pregunta TS2.2 Explica el funcionamiento de cada acción anterior 

Implementa la opción count de otra manera:


*   utilizando transformaciones map y reduce
*   utilizando solo reduce en caso de que sea posible.



# Operaciones K-V (Clave Valor)

In [68]:
import requests
import re
allWords = allWords.flatMap(lambda w: re.sub(""";|:|\.|,|-|–|"|'|\s"""," ", w.lower()).split(" ")).filter(lambda a: len(a)>0)
allWords2 = sc.parallelize(requests.get("https://gist.githubusercontent.com/jsdario/9d871ed773c81bf217f57d1db2d2503f/raw/585de69b0631c805dabc6280506717943b82ba4a/el_quijote_ii.txt").iter_lines())
allWords2 = allWords2.flatMap(lambda w: re.sub(""";|:|\.|,|-|–|"|'|\s"""," ", w.decode("utf8").lower()).split(" ")).filter(lambda a: len(a)>0)

In [ ]:
allWords.take(10)

In [ ]:
allWords2.take(10)

In [ ]:
words = allWords.map(lambda e: (e,1))
words2 = allWords2.map(lambda e: (e,1))

words.take(10)

In [ ]:
frequencies = words.reduceByKey(lambda a,b: a+b)
frequencies2 = words2.reduceByKey(lambda a,b: a+b)
frequencies.takeOrdered(10, key=lambda a: -a[1])

In [ ]:
res = words.groupByKey().takeOrdered(10, key=lambda a: -len(a))
res # To see the content, res[i][1].data

In [ ]:
joinFreq = frequencies.join(frequencies2)
joinFreq.take(10)

In [ ]:
joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: -v[1]), joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: +v[1])

## Pregunta TS2.3 Explica el proposito de cada una de las operaciones anteriores

## Pregunta TS2.4 ¿Cómo puede implementarse la frecuencia con groupByKey y transformaciones?

# Optimizaciones


## Pregunta TS2.5 ¿Cuál de las dos siguientes celdas es más eficiente? Justifique la respuesta.

In [ ]:
joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: -v[1]), joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: +v[1])

In [ ]:
result = joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1])))
result.cache()
result.takeOrdered(10, lambda v: -v[1]), result.takeOrdered(10, lambda v: +v[1])

In [89]:
result.coalesce(numPartitions=2) # Avoids the data movement, so it tries to balance inside each machine
result.repartition(numPartitions=2) # We don't care about data movement, this balance the whole thing to ensure all machines are used

MapPartitionsRDD[182] at coalesce at NativeMethodAccessorImpl.java:0

In [ ]:
result.take(10)
allWords.cache() # allWords RDD must  stay in memory after computation, we made a checkpoint (well, it's a best effort, so must might be too strong)
result.take(10)

In [ ]:
from pyspark import StorageLevel
# https://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#rdd-persistence
allWords2.persist(StorageLevel.MEMORY_AND_DISK) # Now it will be preserved on disk also

In [92]:
!rm -rf palabras_parte2
allWords2.saveAsTextFile("palabras_parte2")

## Pregunta TS2.6 Antes de guardar el fichero, utilice coalesce con diferente valores ¿Cuál es la diferencia?

Variables Globales

In [ ]:
articles = sc.broadcast(["el", "la"])
articles.value

In [ ]:
acc = sc.accumulator(0)
def incrementar(x):
  global acc
  acc += x

allWords.map(lambda l:1).foreach(incrementar)
acc

In [ ]:
print (rdd.getStorageLevel())
rdd = sc.textFile('elquijote.txt')
%time print (rdd.count())
%time print (rdd.count())
rdd.cache()
%time print (rdd.count())
%time print (rdd.count())


In [ ]:
pals_a_eliminar = ['a', 'ante', 'bajo', 'segun', 'que', 'de']

def elimPalabras(p):
    global pals_a_eliminar
    return p not in pals_a_eliminar

lineas = sc.textFile('elquijote.txt', 8)
pals = (lineas.flatMap(lambda linea: linea.lower().split())
              .filter(elimPalabras)
              .map(lambda pal: (pal, 1))
              .reduceByKey(lambda elem1,elem2: elem1 + elem2))

print (pals.takeOrdered(15, key=lambda a:-a[1]))

In [ ]:
counter = sc.accumulator(0)

rdd = sc.textFile('elquijote.txt')

def increment_counter(x):
    global counter
    counter += x
    
rdd.map(lambda l:len(l)).foreach(increment_counter)

print ("Counter value: %d" % counter.value)